<a href="https://colab.research.google.com/github/Akanksha131313/AI_Projects_Journey/blob/main/06_LSTM_Practical_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fake News Classifier Using LSTM -**

Dataset: https://www.kaggle.com/c/fake-news/data#

In [296]:
import pandas as pd

In [297]:
import numpy as np

In [298]:
df = pd.read_csv('/content/sample_data/Fake_news.csv', sep=',', on_bad_lines='skip', engine='python')

In [299]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [300]:
df.shape

(72154, 4)

In [301]:
#Finding NUll values
df.isnull().sum()

,0
Unnamed: 0,0
title,565
text,57
label,20


In [302]:
#Drop NaN Values :
df = df.dropna()

In [303]:
df.shape

(71537, 4)

In [304]:
df = df.reset_index(drop=True)

In [305]:
df

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
...,...,...,...,...
71532,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
71533,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
71534,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
71535,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [306]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [307]:
# here Target ='Label'
# In label , there are mix values , check them first :

df['label'].apply(type).value_counts()

,count
label,
<class 'str'>,71537


In [308]:
import pandas as pd
import numpy as np

# Convert all label values to numeric — invalid ones become NaN
df['label'] = pd.to_numeric(df['label'], errors='coerce')

# Drop rows where label became NaN (non-numeric values)
df = df.dropna(subset=['label'])

# Convert final labels to int
df['label'] = df['label'].astype(int)

# Reset index
df = df.reset_index(drop=True)


/tmp/ipython-input-2626874810.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)


### **Dependent & Independent Features -**

In [309]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [310]:
## Get the Dependent features
y=df['label']

In [311]:
X.shape

(71535, 3)

In [312]:
y.shape

(71535,)

In [313]:
import tensorflow as tf

In [314]:
tf.__version__

'2.19.0'

In [315]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [316]:
### Vocabulary size
voc_size = 500000  # ensure max index fits embedding


### For Onehot Representation , we are doing some basic steps before -

In [317]:
messages=X.copy()

In [318]:
messages=X.copy()

messages.reset_index(inplace=True)

In [319]:
messages['title'][1]

'UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO]'

In [320]:
messages

,index,Unnamed: 0,title,text
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
1,1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
2,2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
3,3,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
4,4,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...,...,...
71530,71530,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...
71531,71531,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n..."
71532,71532,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...
71533,71533,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...


### **Cleaning the Data -**

i) Stemming

ii) Stopwords

In [321]:
import nltk
import re
from nltk.corpus import stopwords

In [322]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [323]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [324]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [325]:
corpus[1]

'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video'

### **One-Hot Representation -**

In [326]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[459422,
  354380,
  169173,
  425569,
  73529,
  76280,
  444016,
  131380,
  449675,
  485967,
  227399,
  68238],
 [247437,
  354268,
  367980,
  338432,
  307542,
  98582,
  25137,
  53230,
  311845,
  453283,
  320877,
  246542,
  278956,
  68238],
 [58663,
  307089,
  162294,
  134233,
  354481,
  322365,
  442893,
  434767,
  265003,
  391958,
  179004,
  158925],
 [425718,
  439502,
  29468,
  123656,
  253521,
  31955,
  9857,
  31384,
  18981,
  45834,
  353990],
 [9584, 442893, 29986, 194534, 215207, 381158, 241703, 182325, 29986],
 [265101,
  117209,
  89311,
  391448,
  19274,
  51221,
  344279,
  392434,
  346115,
  162242,
  206415],
 [123845,
  288138,
  295295,
  423704,
  439502,
  479264,
  322365,
  76669,
  448020,
  68238],
 [255351,
  446531,
  34542,
  139335,
  415565,
  380644,
  338883,
  202856,
  465377,
  255351,
  419147,
  333239,
  205481,
  271724,
  68238],
 [275979, 7183, 276227, 281304, 283798, 316553, 94260, 7183],
 [6101, 371882, 149112, 269640, 

In [327]:
corpus[1]

'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video'

In [328]:
onehot_repr[1]

[247437,
 354268,
 367980,
 338432,
 307542,
 98582,
 25137,
 53230,
 311845,
 453283,
 320877,
 246542,
 278956,
 68238]

### **Embedding Representation -**

In [329]:
sent_length=100
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[459422 354380 169173 ...      0      0      0]
 [247437 354268 367980 ...      0      0      0]
 [ 58663 307089 162294 ...      0      0      0]
 ...
 [336002 378925 450334 ...      0      0      0]
 [423704 221613 432322 ...      0      0      0]
 [196560 355819  91504 ...      0      0      0]]


In [330]:
embedded_docs[1]

array([247437, 354268, 367980, 338432, 307542,  98582,  25137,  53230,
       311845, 453283, 320877, 246542, 278956,  68238,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=int32)

In [331]:
embedded_docs[0]

array([459422, 354380, 169173, 425569,  73529,  76280, 444016, 131380,
       449675, 485967, 227399,  68238,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=int32)

### **LSTM NN -**

In [352]:
## Creating model -> it's super important step
embedding_vector_features=80 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features ))
model.add(Dropout(0.3))
model.add(LSTM(128))# Add return_sequences=True to pass output to the next LSTM layer
model.add(Dropout(0.3)) # Drop-out layer
model.add(Dense(1,activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0) # set your own learning_rate

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


## build the model, provide a dummy shape (just for the summary)

model.build(input_shape=(None, sent_length))

print(model.summary())

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ (None, 100, 80)        │    40,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 80)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 128)            │       107,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,107,137 (153.00 MB)

 Trainable params: 40,107,137 (153.00 MB)

 Non-trainable params: 0 (0.00 B)

None


In [350]:
len(embedded_docs),y.shape

(71535, (71535,))

In [351]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [353]:
X_final.shape,y_final.shape

((71535, 100), (71535,))

In [354]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [355]:
X_train = X_train.astype('float32')

### **Model Training -**

In [339]:
# Early Stoping
#from tensorflow.keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_loss',patience=20,verbose=1,mode='auto',baseline=None, restore_best_weights=False)

In [356]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=100 , callbacks=early_stopping)

Epoch 1/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 19s 35ms/step - accuracy: 0.5074 - loss: 0.6933 - val_accuracy: 0.5071 - val_loss: 0.6930
Epoch 2/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.5143 - loss: 0.6929 - val_accuracy: 0.5071 - val_loss: 0.6931
Epoch 3/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.5111 - loss: 0.6931 - val_accuracy: 0.5071 - val_loss: 0.6932
Epoch 4/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.5134 - loss: 0.6929 - val_accuracy: 0.5071 - val_loss: 0.6931
Epoch 5/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.5081 - loss: 0.6931 - val_accuracy: 0.5071 - val_loss: 0.6931
Epoch 6/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.5112 - loss: 0.6929 - val_accuracy: 0.5071 - val_loss: 0.6931
Epoch 7/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.5115 - loss: 0.6930 - val_accuracy: 0.5071 - val_loss: 0.6931
Epoch 8/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.5146 - loss: 0.6928 - 

### **Performance Metrics And Accuracy -**

In [357]:
y_pred=model.predict(X_test)

738/738 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [358]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve #1 means it's fake & 0 means it's 'real'

In [359]:
from sklearn.metrics import confusion_matrix

In [360]:
confusion_matrix(y_test,y_pred)

array([[11635,     0],
       [11972,     0]])

In [361]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.49286228660990383

In [362]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.49      1.00      0.66     11635
           1       0.00      0.00      0.00     11972

    accuracy                           0.49     23607
   macro avg       0.25      0.50      0.33     23607
weighted avg       0.24      0.49      0.33     23607



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
